In [10]:
import pandas as pd
import numpy as np

In [11]:
# SEE: https://zenodo.org/record/4632335#.YkGzOShBz4c
filename = '../data/GlobalAlienSpeciesFirstRecordDatabase_v2.xlsx'

df_orig = pd.read_excel(filename, sheet_name=None)
# There are 3 sheets of the excel file
print(df_orig.keys())
df_orig = df_orig['GlobalAlienSpeciesFirstRecordDa']
df_orig.info(show_counts=True)



dict_keys(['ReadMe', 'GlobalAlienSpeciesFirstRecordDa', 'References'])
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61751 entries, 0 to 61750
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   TaxonName         61751 non-null  object
 1   scientificName    61751 non-null  object
 2   Family            57925 non-null  object
 3   Order             60709 non-null  object
 4   Class             60664 non-null  object
 5   Phylum            60793 non-null  object
 6   Taxonomy          57045 non-null  object
 7   OrigName          61751 non-null  object
 8   LifeForm          61571 non-null  object
 9   Region            61751 non-null  object
 10  Island            17552 non-null  object
 11  PresentStatus     33473 non-null  object
 12  FirstRecord       61751 non-null  int64 
 13  FirstRecord_orig  61751 non-null  object
 14  Source            61751 non-null  object
dtypes: int64(1), object(14)
memory us

In [12]:
df = df_orig
# Get the correct one
df = df[(df['LifeForm'] != 'Insects') & (df['LifeForm'] != 'Vascular plants') & (df['LifeForm'] != 'Fishes') \
        & (df['LifeForm'] != 'Birds') & (df['LifeForm'] != 'Algae') \
        & (df['LifeForm'] != 'Invertebrates (excl. Arthropods, Molluscs)') 
        & (df['LifeForm'] != 'Molluscs') & (df['LifeForm'] != 'Crustaceans') \
        & (df['LifeForm'] != 'Fungi') & (df['LifeForm'] != 'Reptiles') \
        # & (df['LifeForm'] != 'Arthropods p.p. (Myriapods, Diplopods etc.)') & (df['LifeForm'] != 'Mammals') \
        # & (df['LifeForm'] != 'Amphibians') & (df['LifeForm'] != 'Bryophytes') \
        # & (df['LifeForm'] != 'Amphibians') & (df['LifeForm'] != 'nan') \
        & (df['LifeForm'] != 'Bryozoa') & (df['LifeForm'] != 'Spiders') \
        ]
df = df[(df['FirstRecord'] > 1950) & (df['FirstRecord'] < 2010)]

In [13]:
df.info(show_counts=True)
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1171 entries, 1208 to 61750
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   TaxonName         1171 non-null   object
 1   scientificName    1171 non-null   object
 2   Family            1060 non-null   object
 3   Order             1043 non-null   object
 4   Class             946 non-null    object
 5   Phylum            1040 non-null   object
 6   Taxonomy          1031 non-null   object
 7   OrigName          1171 non-null   object
 8   LifeForm          1023 non-null   object
 9   Region            1171 non-null   object
 10  Island            355 non-null    object
 11  PresentStatus     592 non-null    object
 12  FirstRecord       1171 non-null   int64 
 13  FirstRecord_orig  1171 non-null   object
 14  Source            1171 non-null   object
dtypes: int64(1), object(14)
memory usage: 146.4+ KB


,TaxonName,scientificName,Family,Order,Class,Phylum,Taxonomy,OrigName,LifeForm,Region,Island,PresentStatus,FirstRecord,FirstRecord_orig,Source
1208,Ambystoma jeffersonianum,Ambystoma jeffersonianum,Ambystomatidae,Caudata,Amphibia,Chordata,GBIF,Ambystoma tremblayi,Amphibians,United States,NaN,NaN,1973,1970s,Kraus (2009)
1209,Ambystoma tigrinum,"Ambystoma tigrinum (Green, 1825)",Ambystomatidae,Caudata,Amphibia,Chordata,GBIF,Ambystoma tigrinum,Amphibians,Germany,NaN,casual,1991,1991,DAISIE
1210,Ambystoma tigrinum,"Ambystoma tigrinum (Green, 1825)",Ambystomatidae,Caudata,Amphibia,Chordata,GBIF,Ambystoma tigrinum,Amphibians,Spain,NaN,casual,2002,2002,DAISIE
1212,Anaxyrus americanus,Anaxyrus americanus,Bufonidae,Anura,Amphibia,Chordata,GBIF,Bufo americanus,Amphibians,Canada,NaN,NaN,1960,"1960, 1963, 1964, 1965, 1966",Kraus (2009)
1213,Andrias davidianus,"Andrias davidianus (Blanchard, 1871)",Cryptobranchidae,Caudata,Amphibia,Chordata,GBIF,Andrias davidianus,Amphibians,Japan,yes,alien,1972,1972 - 1972,GISD


In [14]:
family = df['LifeForm'].unique()
region = df['Region'].unique()

print(f'There are {len(family)} families and {len(region)} regions')
species = df['TaxonName'].unique()

print(f'There are {len(species)} species.')
for p in family:
    df_s = df[df['LifeForm'] == p]
    animals = df_s['TaxonName'].unique()
    print(f'{p}[{len(animals)}]', end=', ')


There are 7 families and 159 regions
There are 649 species.
Amphibians[76], Arthropods p.p. (Myriapods, Diplopods etc.)[135], Bacteria and protozoans[27], Bryophytes[91], Mammals[167], Viruses[18], nan[0], 

In [15]:
# TODO This is really really really slow. A better way to do it?
# Filter out irrelevant data

spec_iter = np.zeros(len(species), dtype=int)
reg_iter = np.zeros(len(region), dtype=int)

print('Enumerating species interactions')
for idx, s in enumerate(species):
    df_s = df[df['TaxonName'] == s]
    # n_iter = len(df_s)
    spec_iter[idx] = len(df_s)
    # print(f'The species {s} has {spec_iter[idx]} iteractions')

print('Enumerating region interactions')
for idx, s in enumerate(region):
    df_s = df[df['Region'] == s]
    # n_iter = len(df_s)
    reg_iter[idx] = len(df_s)
    # print(f'The species {s} has {spec_iter[idx]} iteractions')

Enumerating species interactions
Enumerating region interactions


In [16]:
# TODO This is really really really slow. A better way to do it?
# select the species that have less than 3 iteractions
remove_s_idx = np.where(spec_iter < 3)
remove_r_idx = np.where(reg_iter < 10)
# test = df

# Filter the dataset: remove all rows belonging to species with less than 3 iteractions
for s in species[remove_s_idx]:
    # print(s)  
    df = df.drop(df[df['TaxonName'] == s].index)

# Filter the dataset: remove all rows belonging to region with less than 10 invasions
for s in region[remove_r_idx]:
    # print(s)  
    df = df.drop(df[df['Region'] == s].index)

species_removed = len(species) - len(df['TaxonName'].unique())
print(f'Removed {species_removed} species')
region_removed = len(region) - len(df['Region'].unique())
print(f'Removed {region_removed} region')

# Update region, species
region = df['Region'].unique()
species = df['TaxonName'].unique()


Removed 582 species
Removed 132 region


In [17]:
# Buld matrix
n_s = len(species)
n_r = len(region)

t_min = df['FirstRecord'].min()
t_max = df['FirstRecord'].max()
print(t_min, t_max, t_max-t_min)
time_res = 5
time = np.arange(t_min, t_max, time_res)

p = n_s + n_r
M = np.zeros((len(time), n_s, n_r))
# M = np.zeros((len(time), p, p)) # if full

for i, t in enumerate(time):
    df_now = df[(df['FirstRecord'] >= t) & (df['FirstRecord'] < t+(time_res-1))]
    for index, row in df_now.iterrows():
        s = row['TaxonName']
        r = row['Region']
        # print(f'Species {s} invaded region {r} at time {t}')

        s_idx = np.where(species == s)
        r_idx = np.where(region == r)
        M[i, s_idx, r_idx] = 1
M.shape

1951 2007 56


(12, 67, 27)

In [18]:
# Export data and then import it into R
print(n_s, n_r)
M = M.reshape(len(time), n_s*n_r)
# M = M.reshape(len(time), (p)**2) # if full matix
print(f's = {n_s}, r = {n_r}')
print(M.shape)
np.save('../data/matrix.npy', M)

67 27
s = 67, r = 27
(12, 1809)


In [16]:
print(f'There are in total {np.sum(M)} iteraction.')

# This was done for really specific case (only viruses)
# We were studying the results of the R algorithm
idx = np.array([5,  8,  9, 13, 17, 18, 20, 24, 25, 29, 30])
idx = idx - 1
most_relevant_spec = np.sum(M[:, 12, idx])
print(f'Of which {most_relevant_spec} are given by species n 12 {species[12]}')

There are in total 2939.0 iteraction.


IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [11]:
# TODO we should use sparse matrices for the data

import scipy.sparse

rows = [2, 236, 246, 389, 1691]
cols = [117, 3, 34, 2757, 74, 1635, 52]
prod = [(x, y) for x in rows for y in cols] # combinations
r = [x for (x, y) in prod] # x_coordinate
c = [y for (x, y) in prod] # y_coordinate
data = [1] * len(r)
m = scipy.sparse.coo_matrix((data, (r, c)), shape=(100000, 40000))